In [11]:
from scipy.io import wavfile
import torch
import sounddevice as sd
import pytorch_lightning as pl
from tqdm import tqdm
from trainer import Iminegg
from my_iterable_dataset import MyIterableDataset
from im_in_egg_unet1_param import ImineggNet1
from torch.utils.data import DataLoader

C:\Users\alsa0217\Anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [17]:
ds_validate = MyIterableDataset(100, "./data/val22", 22528 * 4)
ds_train = MyIterableDataset(100, "./data/train22", 22528 * 4)

In [19]:
iminegg_net = ImineggNet1(48)
iminegg = Iminegg(iminegg_net)
validate_set = DataLoader(ds_validate, batch_size=20)
train_set = DataLoader(ds_train, batch_size=20)
ckpt = "epoch=503_no_speach_split.ckpt" 
ckpt = "D:\epoch=927_supremacy_of_noise.ckpt"
ckpt = "logs/defaultw48-bs40-drp0.0_relu0.2_add2l/version_3/checkpoints/epoch=797.ckpt"
trainer = pl.Trainer(resume_from_checkpoint=ckpt, gpus=1)
trainer.test(model=iminegg, test_dataloaders=train_set, ckpt_path=ckpt)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


1

In [43]:
samplerate, data = wavfile.read('./tests/source_17.wav')
data = (torch.from_numpy(data).float() / 32768)
sec1 = 11264
off = 0
length = (len(data) - off) // sec1 * sec1

inp = data[off:off + length]

In [44]:
d = next(iter(ds_train))
inp = d['input']
print(d['type'])
device = torch.device("cuda")
samplerate = 22050

2


In [45]:
sd.play(inp.cpu().numpy(), samplerate)

In [46]:
trainer.model.net.eval()
outp = trainer.model.net.calc(inp.reshape(1, 1, -1).to(device))
#outp = trainer.model.net.calc(outp.reshape(1, 1, -1).to(device))

In [47]:
sd.play(outp.cpu().detach().numpy().reshape(-1), samplerate)

In [31]:
export_outp = (outp.cpu() * 32768).short()
export_inp = (inp.cpu() * 32768).short()

wavfile.write('tests/filtered_22khz_21.wav', samplerate, export_outp.detach().numpy())
wavfile.write('tests/source_22khz_21.wav', samplerate, export_inp.detach().numpy())

In [48]:
source = ds_validate
dr = 'data_ft/val'
n = 300

for i in tqdm(range(n)):
    d = next(iter(source))
    inp = d['input']
    trgt = d['target']
    outp = trainer.model.net.calc(inp.reshape(1, 1, -1).to(device))
    export_trgt = (trgt.cpu() * 32768).short()
    export_in = (outp.cpu() * 32768).short()
    wavfile.write('{}/{}_22khz_in_{}.wav'.format(dr, i, d['type']), samplerate, export_in.detach().numpy())
    wavfile.write('{}/{}_22khz_trgt.wav'.format(dr, i), samplerate, export_trgt.detach().numpy())
    
    

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:15<00:00, 18.97it/s]


In [7]:
import re
r = re.match(r'(\d+)_.+?_in_(\d+)\.wav', '13_22khz_in_2.wav')
r[2]

'2'